In [2]:
import pandas as pd
import yfinance as yf
from datetime import date
import datetime
import plotly.express as px
from dateutil.relativedelta import relativedelta
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
import plotly.express as px

C:\Users\Lucas Choconi\AppData\Local\Temp\ipykernel_9896\3956303840.py:9: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Lucas Choconi\AppData\Local\Temp\ipykernel_9896\3956303840.py:10: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
dateToday = date.today()
dateToday = datetime.datetime.strptime(str(dateToday),'%Y-%m-%d').strftime('%Y-%m-%d')
fistDate = date.today()-relativedelta(years=1)
fistDate = datetime.datetime.strptime(str(fistDate),'%Y-%m-%d').strftime('%Y-%m-%d')
fixedTickers = ['ALUA.BA', 'BBAR.BA', 'BMA.BA', 'BYMA.BA', 'CEPU.BA', 'COME.BA', 'CRES.BA', 'EDN.BA', 'GGAL.BA', 'LOMA.BA', 'MIRG.BA',
                'PAMP.BA', 'SUPV.BA', 'TECO2.BA', 'TGNO4.BA', 'TGSU2.BA', 'TRAN.BA', 'TXR.BA', 'VALO.BA', 'YPFD.BA', 'M.BA']

print(fistDate, dateToday)

2022-05-21 2023-05-21


In [4]:
dataGGAL = yf.download(['GGAL.BA', 'GGAL'], start=fistDate, end=dateToday)
histCCL = dataGGAL["Adj Close"].dropna()  
histCCL = histCCL.reset_index() 
histCCL["USD"] = histCCL["GGAL.BA"]*10/histCCL["GGAL"]
histCCL = histCCL.drop(columns=['GGAL','GGAL.BA'])
display ( histCCL.tail(2) )

[*********************100%***********************]  2 of 2 completed


,Date,USD
235,2023-05-18,476.712955
236,2023-05-19,474.224101


In [5]:

dataPanelLider = yf.download(fixedTickers, start=fistDate, end=dateToday)
dataMerval = dataPanelLider["Adj Close"].dropna() 
dataMerval = dataMerval.reset_index()
dataMerval = pd.melt(dataMerval, id_vars='Date',
                        var_name='Ticker', value_name='Close')
dataMerval.tail()

[*********************100%***********************]  21 of 21 completed


,Date,Ticker,Close
5077,2023-05-15,YPFD.BA,5283.950195
5078,2023-05-16,YPFD.BA,5212.850098
5079,2023-05-17,YPFD.BA,5358.850098
5080,2023-05-18,YPFD.BA,5389.450195
5081,2023-05-19,YPFD.BA,5334.299805


In [6]:
dataMerval = pd.DataFrame(pd.merge(histCCL, dataMerval, how='left', on=['Date'])).dropna() 
dataMerval ['Close USD'] = dataMerval['Close']/dataMerval['USD']

dataMerval.tail()

,Date,USD,Ticker,Close,Close USD
4932,2023-05-19,474.224101,TGSU2.BA,1241.650024,2.618277
4933,2023-05-19,474.224101,TRAN.BA,304.500000,0.642101
4934,2023-05-19,474.224101,TXR.BA,9776.500000,20.615781
4935,2023-05-19,474.224101,VALO.BA,97.800003,0.206232
4936,2023-05-19,474.224101,YPFD.BA,5334.299805,11.248479


In [7]:
dataMerval = dataMerval[~((dataMerval.Date=="2022-07-14") & (dataMerval.Ticker=="M.BA"))] ## Filter inaccurate data in Merval

In [8]:
CCL = "Fx Rate (CCL): "+str(round(histCCL["USD"].iloc[-1], ndigits=1))

external_stylesheets = [dbc.themes.ZEPHYR]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app.title = "Merval"
app.layout = html.Div([
                        html.H1(
                            "Historical Data - S&P Merval",
                            style={'text-align': 'left', 'background-color':'#0466C8', 'color':'#EBEBEB', "margin-bottom": "15px" ,  "padding": "10px"}
                            ),

                        html.H2(
                            CCL,
                            style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                            ),
                        
                        dbc.Row(
                            [
                                dbc.Col(
                                    html.H3(
                                    "Select a ticker:",
                                    style={'text-align': 'left', 'color':'#9B9C9B', "margin-bottom": "15px"}
                                    ),
                                    width="auto"),

                                dbc.Col(
                                    dcc.Dropdown(
                                        id="Select_Ticker",                    
                                        options=[{'label': x, 'value': x} for x in fixedTickers],
                                        multi=False,
                                        value='M.BA',
                                        style={'width': "150%", 'color': '#9B9C9B'}
                                        ),
                                    width="auto"
                                    )
                            ],
                        ),



                        dbc.Row(
                                [
                                    dbc.Col(

                                        dcc.Graph(id='lines_plot_USD',config={"displayModeBar": True,
                                                                              'displaylogo': False,
                                                                              'modeBarButtonsToRemove': ['select', 'zoomIn', 'zoomOut', 'autoScale']}, figure={}),
                                        width="auto"),
                                
                                    dbc.Col(
                                        dcc.Graph(id='lines_plot_ARP',config={"displayModeBar": True,
                                                                              'displaylogo': False,
                                                                              'modeBarButtonsToRemove': ['select', 'zoomIn', 'zoomOut', 'autoScale']}, figure={}),
                                        width="auto"),
                                ],
                                justify="evenly",
                                ),
                        ])
                      
                        
@app.callback(
    dash.dependencies.Output('lines_plot_USD', 'figure'),
    dash.dependencies.Output('lines_plot_ARP', 'figure'),
    [dash.dependencies.Input('Select_Ticker', 'value')])

def update_output(value):
    dff = dataMerval[dataMerval["Ticker"] == value]

    fig1 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close USD"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.1f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "USD",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "yaxis": {"tickprefix": "$"},
                                                  "colorway": ["#007BBA"],
                                                  "width": 700,
                                                  "height": 400,
                                              },
                                          }
    fig2 = {
                                              "data": [
                                                  {
                                                      "x": dff["Date"],
                                                      "y": dff["Close"],
                                                      "type": "lines",
                                                      "hovertemplate": "$%{y:.0f}<extra></extra>",
                                                  },
                                              ],
                                              "layout": {
                                                  "title": {
                                                      "text": "ARS (Argentine Peso)",
                                                      "x": 0.05,
                                                      "xanchor": "left",
                                                  },
                                                  "yaxis": {"tickprefix": "$"},
                                                  "colorway": ["#7CCDF4"],
                                                  "width": 700,
                                                  "height": 400,
                                              },
                                          }    

    return fig1, fig2
                              
              
app.run_server(mode='inline')